<a href="https://colab.research.google.com/github/thegreatgupta/ObjectRecognition/blob/master/MobileNet_06_TensorFlow_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [0]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
9.48913026899936
GPU (s):
0.22974575799889863
GPU speedup over CPU: 41x


In [0]:
import tarfile

In [0]:
tar = tarfile.open("object-classification-dataset.tar")

In [0]:
tar.extractall()

In [0]:
tar.close()

In [0]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.layers import Dense,GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [0]:
def model_Xception(img_shape=(299, 299, 3), n_classes=50,
                   load_pretrained=False, freeze_layers_from='base_model'):
    # Decide if load pretrained weights from imagenet
    if load_pretrained:
        weights = 'imagenet'
    else:
        weights = None

    # Get base model
    #base_model = Xception(include_top=False, weights=weights,
                       #input_tensor=None, input_shape=img_shape)
    
    #base_model = ResNet50(include_top=False, weights=weights,
                       #input_tensor=None, input_shape=img_shape)
    
    base_model = MobileNet(include_top=False, weights=weights,
                       input_tensor=None, input_shape=img_shape)

    # Add final layers
    x = base_model.output
    x = Dense(n_classes, activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(n_classes, activation='softmax')(x)

    # This is the model we will train
    model = Model(input=base_model.input, output=predictions)
    
    # Model Summary
    # print(model.summary())
    
    # # let's visualize layer names and layer indices to see how many layers/blocks to re-train
    # # uncomment when choosing based_model_last_block_layer
    # for i, layer in enumerate(model.layers):
    #     print(i, layer.name)

    # Freeze some layers
    if freeze_layers_from is not None:
        if freeze_layers_from == 'base_model':
            print ('   Freezing base model layers')
            for layer in base_model.layers:
                layer.trainable = False
        else:
            for i, layer in enumerate(model.layers):
                print(i, layer.name)
            print ('   Freezing from layer 0 to ' + str(freeze_layers_from))
            for layer in model.layers[:freeze_layers_from]:
               layer.trainable = False
            for layer in model.layers[freeze_layers_from:]:
               layer.trainable = True

    print(model.summary())
    return model

In [0]:
def train_Xception(train_set_path, val_set_path, validation_split = 0.2,
                   batch_size = 32, class_mode = 'categorical', horizontal_flip = False,
                   vertical_flip = False, rotation_range = None, target_size = (299, 299),
                   model = None, epochs = 1):
    
    # Load Image Data Set Using Keras
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = horizontal_flip,
                                   vertical_flip = vertical_flip,
                                   rotation_range=rotation_range,
                                   validation_split=validation_split)
    
    training_set = train_datagen.flow_from_directory(train_set_path,
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 class_mode = class_mode,
                                                 subset = 'training')
    
    validation_set = train_datagen.flow_from_directory(val_set_path,
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 class_mode = class_mode,
                                                 subset = 'validation')
    
    #model.evaluate(training_set)
    #model.evaluate(validation_set)
    
    # Compile Model
    learning_rate = 0.0001
    loss = 'categorical_crossentropy'
    # opt_rms = keras.optimizers.rmsprop(lr=learning_rate,decay=1e-6)
    adam_optimizer = Adam(lr = learning_rate)
    model.compile(optimizer = adam_optimizer, loss = loss, metrics = ['accuracy'])
    
    model.fit_generator(training_set,
                        steps_per_epoch = training_set.samples,
                        epochs = epochs,
                        validation_data = validation_set,
                        validation_steps = validation_set.samples)
    
    #model.evaluate(training_set)
    #model.evaluate(validation_set)
    
    print(model.evaluate_generator(training_set, 49))

    print(model.evaluate_generator(validation_set, 49))
    
    return model

In [0]:
# Declare Constants
img_shape = (128, 128, 3)
target_size = (128, 128)
n_classes = 49
epochs = 3
train_set_path = '101_ObjectCategories' #'dataset2/training_set_test'
val_set_path = train_set_path
validation_split = 0.15
batch_size = 64
horizontal_flip = True
rotation_range = None
learning_rate = 0.0001
loss = 'categorical_crossentropy'

In [0]:
# Build Model
model = model_Xception(img_shape = img_shape,  n_classes = n_classes, load_pretrained = True)

   Freezing base model layers
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
Epoch 1/10
4584/4584 [==============================] - 577s 126ms/step - loss: 0.7539 - acc: 0.8199 - val_loss: 0.3317 - val_acc: 0.8983
Epoch 2/10
4584/4584 [==============================] - 579s 126ms/step - loss: 0.1453 - acc: 0.9640 - val_loss: 0.3118 - val_acc: 0.9010
Epoch 3/10
4584/4584 [==============================] - 580s 127ms/step - loss: 0.0815 - acc: 0.9799 - val_loss: 0.2787 - val_acc: 0.9092
Epoch 4/10
4584/4584 [==============================] - 577s 126ms/step - loss: 0.0574 - acc: 0.9856 - val_loss: 0.2587 - val_acc: 0.9191
Epoch 5/10
4584/4584 [==============================] - 582s 127ms/step - loss: 0.0442 - acc: 0.9891 - val_loss: 0.2446 - val_acc: 0.9248
Epoch 6/10
4584/4584 [==============================] - 578s 126ms/step - loss: 0.0360 - acc: 0.9907 - val_loss: 0.2824 - val_acc: 0.9129
Epoch 7/10
4584/4584 [==============================] - 572s 125ms/step - loss: 0.0298 

In [0]:
model.save_weights('obj_recognition_xception_normal_bt16_ep10.h5')

In [0]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4850 images belonging to 49 classes.
Found 516 images belonging to 49 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
4850/4850 [==============================] - 977s 201ms/step - loss: 0.5295 - acc: 0.8756 - val_loss: 0.3464 - val_acc: 0.8935
Epoch 2/10
4850/4850 [==============================] - 977s 201ms/step - loss: 0.0668 - acc: 0.9858 - val_loss: 0.3013 - val_acc: 0.9066
Epoch 3/10
4850/4850 [==============================] - 968s 200ms/step - loss: 0.0325 - acc: 0.9936 - val_loss: 0.3336 - val_acc: 0.8991
Epoch 4/10
4850/4850 [==============================] - 955s 197ms/step - loss: 0.0203 - acc: 0.9960 - val_loss: 0.3026 - val_acc: 0.9078
Epoch 5/10
4850/4850 [==============================] - 942s 194ms/step - loss: 0.0150 - acc: 0.9970 - val_loss: 0.3152 - val_acc: 0.9072
Epoch 6/10
4850/4850 [==============================] - 954s 197ms/step - loss: 0.0119 - acc: 0.9975 - val_loss: 0.2990 - val_acc: 0.9158
Epoch 7/10
4850/4850 [=================

In [15]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
4446/4584 [============================>.] - ETA: 14s - loss: 0.3733 - acc: 0.9045Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
4584/4584 [==============================] - 539s 118ms/step - loss: 0.3651 - acc: 0.9066 - val_loss: 0.3101 - val_acc: 0.9080
Epoch 2/10
4584/4584 [==============================] - 535s 117ms/step - loss: 0.0641 - acc: 0.9822 - val_loss: 0.2698 - val_acc: 0.9251
Epoch 3/10
4584/4584 [==============================] - 533s 116ms/step - loss: 0.0422 - acc: 0.9875 - val_loss: 0.3375 - val_acc: 0.9080
Epoch 4/10
4584/4584 [==============================] - 534s 116ms/step - loss: 0.0322 - acc: 0.9905 - val_loss: 0.2718 - val_acc: 0.9238
Epoch 5/10
4584/4584 [==============================] - 533s 116ms/step - loss: 0.0274 - 

In [0]:
model.save_weights('obj_recognition_mobilenet_00001_015_bt64_ep3.h5')

In [0]:
model.save('mobilenet_00001_015_bt64_ep3_model02.h5')

In [0]:
model.load_weights('obj_recognition_mobilenet_00001_015_bt64_ep3.h5')

In [0]:
json_string = model.to_json()

In [77]:
json_string

'{"class_name": "Model", "config": {"name": "model_1", "layers": [{"name": "input_1", "class_name": "InputLayer", "config": {"batch_input_shape": [null, 128, 128, 3], "dtype": "float32", "sparse": false, "name": "input_1"}, "inbound_nodes": []}, {"name": "conv1_pad", "class_name": "ZeroPadding2D", "config": {"name": "conv1_pad", "trainable": false, "padding": [[0, 1], [0, 1]], "data_format": "channels_last"}, "inbound_nodes": [[["input_1", 0, 0, {}]]]}, {"name": "conv1", "class_name": "Conv2D", "config": {"name": "conv1", "trainable": false, "filters": 32, "kernel_size": [3, 3], "strides": [2, 2], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1, 1], "activation": "linear", "use_bias": false, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regulariz

In [0]:
f = open('json_00001_64_model.txt', 'w+')

In [94]:
f.write(json_string)

43142

In [0]:
f.close()

In [28]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
Epoch 1/5
4584/4584 [==============================] - 546s 119ms/step - loss: 0.0170 - acc: 0.9945 - val_loss: 0.3994 - val_acc: 0.9089
Epoch 2/5
4584/4584 [==============================] - 543s 118ms/step - loss: 0.0160 - acc: 0.9950 - val_loss: 0.3108 - val_acc: 0.9261
Epoch 3/5
4584/4584 [==============================] - 542s 118ms/step - loss: 0.0150 - acc: 0.9951 - val_loss: 0.2867 - val_acc: 0.9300
Epoch 4/5
4584/4584 [==============================] - 543s 118ms/step - loss: 0.0141 - acc: 0.9954 - val_loss: 0.3657 - val_acc: 0.9169
Epoch 5/5
4584/4584 [==============================] - 550s 120ms/step - loss: 0.0132 - acc: 0.9959 - val_loss: 0.3627 - val_acc: 0.9194


In [42]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
Epoch 1/5
4584/4584 [==============================] - 556s 121ms/step - loss: 0.0095 - acc: 0.9967 - val_loss: 0.3247 - val_acc: 0.9281
Epoch 2/5
4584/4584 [==============================] - 557s 121ms/step - loss: 0.0083 - acc: 0.9974 - val_loss: 0.3650 - val_acc: 0.9181
Epoch 3/5
4584/4584 [==============================] - 555s 121ms/step - loss: 0.0081 - acc: 0.9974 - val_loss: 0.3128 - val_acc: 0.9324
Epoch 4/5
4584/4584 [==============================] - 553s 121ms/step - loss: 0.0072 - acc: 0.9976 - val_loss: 0.3127 - val_acc: 0.9302
Epoch 5/5
4584/4584 [==============================] - 551s 120ms/step - loss: 0.0077 - acc: 0.9972 - val_loss: 0.3037 - val_acc: 0.9329


In [52]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
Epoch 1/5
4584/4584 [==============================] - 553s 121ms/step - loss: 0.0069 - acc: 0.9979 - val_loss: 0.3276 - val_acc: 0.9283
Epoch 2/5
4584/4584 [==============================] - 539s 117ms/step - loss: 0.0068 - acc: 0.9978 - val_loss: 0.3214 - val_acc: 0.9290
Epoch 3/5
4584/4584 [==============================] - 546s 119ms/step - loss: 0.0069 - acc: 0.9978 - val_loss: 0.3327 - val_acc: 0.9292
Epoch 4/5
4584/4584 [==============================] - 550s 120ms/step - loss: 0.0074 - acc: 0.9977 - val_loss: 0.3388 - val_acc: 0.9240
Epoch 5/5
4584/4584 [==============================] - 552s 120ms/step - loss: 0.0066 - acc: 0.9981 - val_loss: 0.3053 - val_acc: 0.9334


In [68]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
Epoch 1/5
4584/4584 [==============================] - 315s 69ms/step - loss: 0.0336 - acc: 0.9893 - val_loss: 0.3404 - val_acc: 0.9302
Epoch 2/5
4584/4584 [==============================] - 315s 69ms/step - loss: 0.0343 - acc: 0.9897 - val_loss: 0.3025 - val_acc: 0.9304
Epoch 3/5
4584/4584 [==============================] - 315s 69ms/step - loss: 0.0299 - acc: 0.9903 - val_loss: 0.3506 - val_acc: 0.9242
Epoch 4/5
4584/4584 [==============================] - 314s 69ms/step - loss: 0.0324 - acc: 0.9894 - val_loss: 0.3764 - val_acc: 0.9216
Epoch 5/5
4584/4584 [==============================] - 314s 69ms/step - loss: 0.0284 - acc: 0.9905 - val_loss: 0.3608 - val_acc: 0.9228


In [71]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
Epoch 1/5
4584/4584 [==============================] - 1013s 221ms/step - loss: 0.0027 - acc: 0.9990 - val_loss: 0.3034 - val_acc: 0.9353
Epoch 2/5
4584/4584 [==============================] - 1007s 220ms/step - loss: 0.0031 - acc: 0.9989 - val_loss: 0.2899 - val_acc: 0.9362
Epoch 3/5
4584/4584 [==============================] - 1006s 219ms/step - loss: 0.0029 - acc: 0.9990 - val_loss: 0.3120 - val_acc: 0.9304
Epoch 4/5
4584/4584 [==============================] - 1009s 220ms/step - loss: 0.0028 - acc: 0.9990 - val_loss: 0.3168 - val_acc: 0.9283
Epoch 5/5
4584/4584 [==============================] - 1011s 221ms/step - loss: 0.0029 - acc: 0.9990 - val_loss: 0.3087 - val_acc: 0.9351


In [83]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
[0.026233947051729715, 0.9885204081632653]
[0.3375312190983565, 0.9361290322580645]


In [86]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
Epoch 1/3
4584/4584 [==============================] - 1936s 422ms/step - loss: 0.0019 - acc: 0.9992 - val_loss: 0.3447 - val_acc: 0.9283
Epoch 2/3
4584/4584 [==============================] - 1933s 422ms/step - loss: 0.0018 - acc: 0.9992 - val_loss: 0.3229 - val_acc: 0.9283
Epoch 3/3
4584/4584 [==============================] - 1940s 423ms/step - loss: 0.0017 - acc: 0.9992 - val_loss: 0.3191 - val_acc: 0.9294
[0.04359548127222854, 0.9853316326530612]
[0.29695101992973527, 0.9366485011187821]


In [0]:
model = train_Xception(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
                       batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
                       model = model, epochs = epochs, target_size = target_size)

Found 4584 images belonging to 49 classes.
Found 782 images belonging to 49 classes.
Epoch 1/3
 253/4584 [>.............................] - ETA: 27:17 - loss: 0.0020 - acc: 0.9992